In [34]:
import pandas as pd

In [35]:
songs = pd.read_csv("D:\\Projects\\airflow_docker\\data\\songs.csv")
streams = pd.read_csv("D:\\Projects\\airflow_docker\\data\\streams1.csv")

In [36]:
#correct streams datetime to date
streams['listen_date'] = pd.to_datetime(streams['listen_time']).dt.date

In [37]:
#join streams and songs using track_id
merged_data = streams.merge(songs, on = 'track_id', how = 'left')

In [41]:
# Compute each KPI
# KPI 1: Daily Genre Listen Count
genre_listen_count = merged_data.groupby(['listen_date', 'track_genre']).size().reset_index(name = 'listen_count')

In [48]:
# KPI 2: Average Listening Duration per Genre per Day
merged_data['duration_seconds'] = merged_data['duration_ms']/1000
avg_duration  = merged_data.groupby(['listen_date', 'track_genre'])['duration_seconds'].mean().reset_index(name = 'average_duration')

In [57]:
# KPI 3: Daily Genre Popularity Index
total_listens = merged_data.groupby('listen_date').size().reset_index(name = 'total_listens')
genre_listen_count = genre_listen_count.merge(total_listens, on = 'listen_date', how = 'left')
genre_listen_count['popularity_index'] = genre_listen_count['listen_count']/ genre_listen_count['total_listens']

In [71]:
# KPI 4: Most Popular Track per Genre per Day
most_popular_track = merged_data.groupby(['listen_date', 'track_genre', 'track_id']).size().reset_index(name='track_count')
most_popular_track = most_popular_track.sort_values(['listen_date', 'track_genre', 'track_id'], ascending= [True,True,False])
most_popular_track = most_popular_track.drop_duplicates(subset=['listen_date', 'track_genre'],keep = 'first').rename(columns = {'track_id' : 'most_popular_track_id'})

In [76]:
# Combine all KPIs into one DataFrame
final_kpis = genre_listen_count[['listen_date','track_genre','listen_count','popularity_index']]
final_kpis = final_kpis.merge(avg_duration, on =  ['listen_date','track_genre'])
final_kpis.merge(most_popular_track, on 

,listen_date,track_genre,listen_count,popularity_index,average_duration
0,2024-06-25,acoustic,118,0.010400,215.379034
1,2024-06-25,afrobeat,124,0.010929,248.978960
2,2024-06-25,alt-rock,119,0.010488,236.073731
3,2024-06-25,alternative,45,0.003966,202.768200
4,2024-06-25,ambient,119,0.010488,255.296067
...,...,...,...,...,...
108,2024-06-25,techno,46,0.004054,296.812717
109,2024-06-25,trance,97,0.008549,298.194134
110,2024-06-25,trip-hop,122,0.010753,263.405664
111,2024-06-25,turkish,112,0.009871,223.225420
